In [128]:
import pandas as pd

from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from sklearn.feature_selection import GenericUnivariateSelect, chi2



from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from scipy import sparse

In [177]:
class MyBinaryRelevanceFeatureSelect():
   
    def fit(self, X, y):
        
        # I'm using a gaussian naive bayes base classifier
        self.BinaryRelevanceObject = BinaryRelevance(classifier = SVC(gamma= 'auto'), require_dense = [True, True])
        
        # fitting the data
        self.BinaryRelevanceObject.fit(X, y)
        
        #the classifiers for each label
        self.classifiers = self.BinaryRelevanceObject.classifiers_

        return self.classifiers
        
    def partition(self):
        return self.BinaryRelevanceObject.partition_#BinaryRelevanceObject
    
    def model_count(self):
        return self.BinaryRelevanceObject.model_count_

    
    def predict(self, X, y=None):
        return self.BinaryRelevanceObject.predict(X)
    
    def feature_select(self, X, y):
        
        features_selected = []

#         X_new = SelectKBest(chi2, k=2)

#         # the feature selecting
#         X_new.fit(X, y)

#         # save indices of the saved attributes
#         selected_attributes_indices = X_new.get_support(indices = True)
        
        transformer = GenericUnivariateSelect(chi2, 'k_best', param=2)
        X_new = transformer.fit_transform(X, y)
        X_new.shape
        selected_attributes_indices = transformer.get_support(indices = True)
        
        return selected_attributes_indices

In [178]:
clf.feature_select(X_test,y_test)

array([ 2,  7,  8, 16, 19], dtype=int64)

In [179]:
#making the dataset
X, y = make_multilabel_classification(n_classes=6, n_labels=2,sparse = True, allow_unlabeled=False, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [180]:
#split in training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [181]:
clf = MyBinaryRelevanceFeatureSelect()
clf

In [182]:
clf.fit(X_train, y_train)

[SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_s

In [183]:
predictions = clf.predict(X_test)
predictions[0]

<1x6 sparse matrix of type '<class 'numpy.int32'>'
	with 1 stored elements in Compressed Sparse Column format>

In [184]:
x = pd.DataFrame(predictions.toarray()).head()

In [185]:
x[0]

0    0
1    0
2    0
3    0
4    0
Name: 0, dtype: int32

In [186]:
pd.DataFrame(y_test).head()

,0,1,2,3,4,5
0,1,1,0,1,0,0
1,0,1,0,0,0,0
2,1,1,0,0,1,0
3,0,1,0,1,1,0
4,1,0,0,0,0,0


In [187]:
clf.feature_select(X_test,predictions)

array([18, 19], dtype=int64)

In [188]:
clf.classifiers

[SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False),
 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_s

In [189]:
clf.partition()

[0, 1, 2, 3, 4, 5]

In [190]:
clf.model_count()

6

In [191]:
clf.feature_select(X_test,predictions)

array([18, 19], dtype=int64)

In [214]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC

        # initialize Binary Relevance multi-label classifier
        # with an SVM classifier
        # SVM in scikit only supports the X matrix in sparse representation

classifier = BinaryRelevance(
            classifier = SVC(),
            require_dense = [False, True]
        )

        # train
classifier.fit(X_train, y_train)

        # predict
predictions = classifier.predict(X_test)

C:\Users\Abel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [215]:
pd.DataFrame(predictions.toarray()).head()

,0,1,2,3
0,1,1,0,1
1,1,1,0,1
2,1,1,0,1
3,1,1,0,1
4,1,1,0,0


In [216]:
pd.DataFrame(y_train).head()

,0,1,2,3
0,1,0,0,0
1,0,1,0,0
2,1,1,1,1
3,0,1,0,0
4,0,1,0,0


In [217]:
classifier.partition_

[0, 1, 2, 3]

In [218]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
iris = load_iris()
X, y = iris.data, iris.target
print(X.shape)
print(y.shape)


lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
print(X_new.shape)

(150, 4)
(150,)
(150, 3)


C:\Users\Abel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
